In [153]:
from pathlib import Path
from wayve.core.calibration.load_calibration import load_calibration_textproto
from infrastructure.robot.fleet import get_fleet_data
from google.protobuf.text_format import MessageToString
from google.protobuf.json_format import MessageToDict
import json

VEHICLES = list(get_fleet_data().keys())
VEHICLES = ["po"]

REPO_ROOT = "/home/benjin/Development/wcs/WayveCode"
CALIBRATION_DIR = Path(REPO_ROOT) / "infrastructure" / "robot" / "config"

OUTPUT_DIR = Path("/mnt/remote/data/benjin/tmp_share/notebooks/out/align_calib_calib_prior")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [154]:
def get_calibration_gen1(vehicle_dir: Path, prior: bool):
    if prior:
        textproto_file = vehicle_dir / "bb" / "vehicle_calibration_default.textproto"

    else:
        textproto_file = vehicle_dir / "bb" / "vehicle_calibration.textproto"

    calib = load_calibration_textproto(textproto_file)
    calib = convert_to_v2(calib)
    
    return calib

for vehicle in VEHICLES:
    try:
        vehicle_dir = CALIBRATION_DIR / vehicle
        calib = get_calibration_gen1(vehicle_dir, prior=False)
        calib_prior = get_calibration_gen1(vehicle_dir, prior=True)

        # Sort for easier comparison
        calib_dict = MessageToDict(calib)
        calib_prior_dict = MessageToDict(calib_prior)
        calib_dict["sensorCalibrations"] = {key: calib_dict["sensorCalibrations"][key] for key in sorted(calib_dict["sensorCalibrations"].keys())}
        calib_prior_dict["sensorCalibrations"] = {key: calib_prior_dict["sensorCalibrations"][key] for key in sorted(calib_prior_dict["sensorCalibrations"].keys())}

        output_dir = OUTPUT_DIR / vehicle
        output_dir.mkdir(parents=True, exist_ok=True)
        json.dump(calib_dict, open(output_dir / "vehicle_calibration.json", "w"), indent=4)
        json.dump(calib_prior_dict, open(output_dir / "vehicle_calibration_default.json", "w"), indent=4)
        print(f"Success for {vehicle}")
    except Exception as e:
        print(f"Failed for {vehicle}: {e}")
        continue

Failed for po: [Errno 2] No such file or directory: '/home/benjin/Development/wcs/WayveCode/infrastructure/robot/config/po/bb/vehicle_calibration.textproto'
